In [ ]:
import pickle
import pandas as pd
import numpy as np
import time
import math
import os
from PIL import Image
from sklearn.model_selection import train_test_split

import datetime
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim


a1dir = "../../../scratch/lt2326-h21/a1"
imgs = os.listdir(a1dir+'/images')

batch_size = 4

In [ ]:
def get_meta(direc):
    
    df = pd.read_json(direc + "/train.jsonl", lines=True)
    imgs = os.listdir(direc+'/images')
    #print(df.columns)
    df =  df[df['file_name'].isin(imgs)]
    df = df.drop(['ignore', 'image_id', 'height', 'width'], axis=1)    
    return df

In [ ]:
df = get_meta(a1dir)

In [ ]:
# this literally doesn't even take a second so no worries
def get_bboxes(df, k=None):
    
    if k is not None:
        df = df.sample(k, random_state=42)
    
    all_bboxes = {}
    for file in df.iterrows():
        img_bboxes = [] # bboxes per image
        for sign in file[1]['annotations']:
            for d in sign:
                if d['is_chinese']:
                    x, y, w, h = d['adjusted_bbox']
                    xmin = int(math.floor(x))
                    xmax = int(math.ceil(x + w))
                    ymin = int(math.floor(y))
                    ymax = int(math.ceil(y + h))
                    bbox = np.array([xmin, ymin, xmax, ymax])
                    img_bboxes.append(bbox)


        all_bboxes[file[1]['file_name']] = np.array(img_bboxes)
    
    return all_bboxes
    

In [ ]:
tic = time.perf_counter()
all_bboxes = get_bboxes(df)
toc = time.perf_counter()
print(f"Finished in {toc - tic:0.4f} seconds")

In [ ]:
def get_labels(bboxes=all_bboxes):
    
    print(f'Time now: {datetime.datetime.now().strftime("%H:%M:%S")}')
    
    p =  [(img_i0, img_i1) for img_i0, img_i1 in np.ndindex(2048, 2048)] # first two numbers of indices of each image

    tic = time.perf_counter()
    labelling = {}
    count = 0
    for fn, bbox in all_bboxes.items():
        tac = time.perf_counter()
        lab = ((p>=bbox[:,None,:2]) & (p<=bbox[:,None,2:])).all(2) # thanks https://stackoverflow.com/questions/62235257/determine-if-many-points-are-in-bounding-box
        lab2 = lab.reshape(len(bbox), 2048, 2048).astype(int)
        #lab2int = lab2.astype(int)
        lab2intsumclip = np.clip(np.sum(lab2, axis = 0), 0, 1).reshape(2048, 2048, 1)
        #lab2intsumreshape = lab2intsumclip.reshape(2048, 2048, 1)

        labelling[fn] = lab2intsumclip
        count += 1
        tuc = time.perf_counter()
        print(f"Finished image {fn} in {tuc - tac:0.4f} seconds, {count}/{len(all_bboxes)}")


    toc = time.perf_counter()
    final_time = str(datetime.timedelta(seconds=round(toc - tic)))
    s_per_img = (toc - tic)/len(all_bboxes)
    
    print(f"Finished in {final_time}\nAvg. time per img: {s_per_img} seconds.")
    print(f'Finished at: {datetime.datetime.now().strftime("%H:%M:%S")}')
    
    return labelling


In [ ]:

#labelling = get_labels()

In [ ]:
#pickle.dump(labelling, open("labelling_alsotrash.p", "wb"))

## Start running here if you have labels

In [ ]:
labelling = pickle.load(open("labelling_small.p", "rb"))

In [ ]:
file_names = labelling.keys() 

In [ ]:
def get_imgs(filenames, datadir, imgs):
    return {x : np.array(Image.open("{}/{}".format(datadir, x)).convert('RGB')) for x in filenames if x in imgs}

In [ ]:
data = get_imgs(file_names, a1dir+'/images', imgs)

In [ ]:
label_arrs = list(labelling.values())

In [ ]:
img_arrs = list(data.values())

## PART 2

In [ ]:

device = torch.device('cuda:2')

# training hyperparameters
batch_size = 4
learning_rate = 0.01
epochs = 3

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(img_arrs, label_arrs, train_size=0.8, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, train_size=0.5, random_state=42)

In [ ]:
dataload_train = DataLoader(list(zip(X_train, y_train)), batch_size=batch_size, shuffle=True)
dataload_test = DataLoader(list(zip(X_test, y_test)), batch_size=batch_size, shuffle=True)
dataload_val = DataLoader(list(zip(X_val, y_val)), batch_size=batch_size, shuffle=True)


In [ ]:
class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()
    
    def forward(self, x):
        print(x.shape)
        return x


class Lenet(nn.Module):
    def __init__(self):
        super(Lenet, self).__init__()
        
        self.cnn = nn.Sequential(            
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),  # [B, 3, 2048, 2048] -> [B, 6, 2044, 2044]
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2),                              # [B, 6, 2044, 2044] -> [B, 6, 681, 681]
            
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1), # [B, 6, 681, 681] -> [B, 16, 677, 677]

            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2),                              # [B, 16, 677, 677] -> [B, 16, 225, 225]
            
            nn.Conv2d(in_channels=16, out_channels=12, kernel_size=5),          # [B, 16, 225, 225] -> [B, 12, 504, 504]
            nn.Tanh(),            
            nn.AvgPool2d(kernel_size=3),                                        # [B, 12, 504, 504] -> [B, 12, 168, 168]
            PrintLayer(),            
        )
        
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=505*505*12, out_features=500),
            nn.ReLU(),
            nn.Linear(in_features=500, out_features=500//2),
            nn.Sigmoid()
        )        
    
    def forward(self, x):
        x = self.cnn(x) 
        #print(x.shape)
        output = self.fc(x)
        return output

In [ ]:


def train(model, dataloader, model_num, device=torch.device('cuda:2'), lr=0.01, e=5):    
    
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()
    model.train()
    
    upsample = nn.Upsample(size=2048*2048)
    
    print('Training')
    for e in range(e):
        total_loss = 0
        for i , data in enumerate(dataloader):
            
            X, y = data
            X = X.permute(0,3,1,2).float()
            X, y = (X.to(device), y.to(device))
            output = model(X)
            
            
            upsampled = upsample(output.unsqueeze(1))
        
            
            loss = criterion(upsampled.reshape(y.shape), y.float())
            
            total_loss += loss.item()
            print(total_loss/(i+1), end='\r')            
            loss.backward() 
            optimizer.step() 
            optimizer.zero_grad()
        print()
    print(f'Saving model{model_num}')
    # save model
    #torch.save(model, f"model{model_num}.pt")

leNet = Lenet()
train(leNet, dataload_train, '1')